# Plan

- 







In [ ]:
!pip install bpe

In [15]:
# imports


from google.colab import files
import pandas as pd
import io
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np
import string
from keras.utils import to_categorical
from bpe import Encoder
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [17]:
# load data
uploaded = files.upload()

Saving babynames-clean.csv to babynames-clean.csv


In [18]:
names = pd.read_csv(io.BytesIO(uploaded['babynames-clean.csv']))
names.drop(names.columns[[1, 1]], axis=1, inplace=True) # drop genders

In [20]:
##########################
#         BPE            #
##########################

corpus = names["John"].tolist() # to list ify

encoder = Encoder(800, pct_bpe=1)  # encode it all
encoder.fit(corpus) # fit it to the corpus

names["tokenized_names"] = names.apply(lambda row:encoder.tokenize(row["John"]), axis=1) # make the collumn

names.head()


John                tokenized_names
0  William  [__sow, wi, ll, ia, m, __eow]
1    James      [__sow, ja, me, s, __eow]
2  Charles  [__sow, ch, ar, le, s, __eow]
3   George     [__sow, ge, or, ge, __eow]
4    Frank      [__sow, fr, an, k, __eow]

In [21]:
## find max_len

high = 0 # initialize high
high = names.apply(lambda row: max(high, len(row["tokenized_names"])), axis=1) # jank, could cut the max func here
max_len = max(high.tolist()) # take the highest len value
print(max_len)


8


In [22]:
##########################
#       Padding          #
##########################

names["padded"] = names.apply(lambda row: row["tokenized_names"]+(max_len - len(row["tokenized_names"]))*["__pad"], axis=1) # pad em

names.head()


John  ...                                           padded
0  William  ...      [__sow, wi, ll, ia, m, __eow, __pad, __pad]
1    James  ...   [__sow, ja, me, s, __eow, __pad, __pad, __pad]
2  Charles  ...      [__sow, ch, ar, le, s, __eow, __pad, __pad]
3   George  ...  [__sow, ge, or, ge, __eow, __pad, __pad, __pad]
4    Frank  ...   [__sow, fr, an, k, __eow, __pad, __pad, __pad]

[5 rows x 3 columns]

In [23]:
##########################
#       Tokenize         #
##########################

tok = defaultdict(lambda: len(tok)) # define the default dict. add one to deal with new keys

names["toked"] = [[tok[j] for j in i] for i in names["padded"].tolist()] # tokenize it!

names.head()


John  ...                        toked
0  William  ...     [0, 1, 2, 3, 4, 5, 6, 6]
1    James  ...     [0, 7, 8, 9, 5, 6, 6, 6]
2  Charles  ...  [0, 10, 11, 12, 9, 5, 6, 6]
3   George  ...  [0, 13, 14, 13, 5, 6, 6, 6]
4    Frank  ...  [0, 15, 16, 17, 5, 6, 6, 6]

[5 rows x 4 columns]

In [24]:
##########################
#       One Hot          #
##########################

names["one_hot"] = names.apply(lambda row: to_categorical(row["toked"], num_classes=len(tok)), axis=1) # one hot it! num_classes is len of tok 

names.head()

John  ...                                            one_hot
0  William  ...  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
1    James  ...  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
2  Charles  ...  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
3   George  ...  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
4    Frank  ...  [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...

[5 rows x 5 columns]

In [45]:
#####################
#   df to tensor    #
#####################

# fin = names["one_hot"].astype("float32")

# print(fin)

fin = np.array(names["one_hot"].tolist())

tnames = torch.from_numpy(fin)

print(tnames)

# np_array = fin.to_numpy(dtype="float32")
# np_array = np_array.astype('float64')

# print(np_array)

# tnames = torch.from_numpy(np_array)

# print(np_array)


tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

In [6]:
###################################
#     Model :clap: Time :clap:    #
###################################


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # layers

    def forward(self, x):
        # model structure
        # x = 
        return x


net = Net()






In [8]:
###################
#   OPTIMIZER     #
###################


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)





ValueError: ignored

In [ ]:
###################
#     TRAINING    #
###################

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        name = 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


